In [1]:
import pandas as pd
from dateutil.parser import parse as dateParse

In [2]:
df = pd.read_csv('./Data/listings-2019-08-06.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,46,2019-07-14,0.39,2,288
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,274,2019-07-26,4.64,1,212
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,75,2019-07-21,0.60,1,127


In [3]:
df = df.drop(['id', 'name', 'host_id', 'host_name'], axis=1)
df.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,46,2019-07-14,0.39,2,288
1,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,274,2019-07-26,4.64,1,212
3,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,75,2019-07-21,0.60,1,127


In [4]:
df.insert(0, 'date', dateParse('2019-08-06'), True)
df.head()

,date,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2019-08-06,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,46,2019-07-14,0.39,2,288
1,2019-08-06,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,2019-08-06,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,274,2019-07-26,4.64,1,212
3,2019-08-06,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,2019-08-06,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,75,2019-07-21,0.60,1,127


In [5]:
def parseThisBoi(x):
    if(pd.isna(x)):
        return None
    return dateParse(x)
df['last_review'] = df['last_review'].apply(parseThisBoi)
df['last_review'] = df['date'] - df['last_review']

In [6]:
df = df.rename(columns={"last_review": "days_since_last_review"})
df.head()

,date,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,days_since_last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2019-08-06,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,46,23 days,0.39,2,288
1,2019-08-06,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaT,NaN,1,365
2,2019-08-06,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,274,11 days,4.64,1,212
3,2019-08-06,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,260 days,0.10,1,0
4,2019-08-06,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,75,16 days,0.60,1,127


In [7]:
def refineDataFrame(df, date):
    df = df.drop(['id', 'name', 'host_id', 'host_name'], axis=1)
    df.insert(0, 'date', dateParse(date), True)  
    df['last_review'] = df['last_review'].apply(parseThisBoi)
    df['last_review'] = df['date'] - df['last_review']
    return df.rename(columns={"last_review": "days_since_last_review"})  

In [8]:
df2 = pd.read_csv('./Data/listings-2018-08-06.csv')
df2 = refineDataFrame(df2, "2018-08-06")
df3 = pd.read_csv('./Data/listings-2017-08-02.csv')
df3 = refineDataFrame(df3, "2017-08-02")

In [13]:
data = pd.concat([df,df2,df3],ignore_index=True)

In [14]:
data

,date,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,days_since_last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2019-08-06,Manhattan,Midtown,40.753620,-73.983770,Entire home/apt,225,1,46,23 days,0.39,2,288
1,2019-08-06,Manhattan,Harlem,40.809020,-73.941900,Private room,150,3,0,NaT,NaN,1,365
2,2019-08-06,Brooklyn,Clinton Hill,40.685140,-73.959760,Entire home/apt,89,1,274,11 days,4.64,1,212
3,2019-08-06,Manhattan,East Harlem,40.798510,-73.943990,Entire home/apt,80,10,9,260 days,0.10,1,0
4,2019-08-06,Manhattan,Murray Hill,40.747670,-73.975000,Entire home/apt,200,3,75,16 days,0.60,1,127
5,2019-08-06,Brooklyn,Bedford-Stuyvesant,40.686880,-73.955960,Private room,60,45,49,670 days,0.39,1,0
6,2019-08-06,Manhattan,Hell's Kitchen,40.764890,-73.984930,Private room,79,2,434,24 days,3.48,1,239
7,2019-08-06,Manhattan,Upper West Side,40.801780,-73.967230,Private room,79,2,118,746 days,0.98,1,0
8,2019-08-06,Manhattan,East Village,40.727640,-73.979490,Entire home/apt,116,30,94,1147 days,0.74,1,322
9,2019-08-06,Manhattan,Chinatown,40.713440,-73.990370,Entire home/apt,150,1,161,8 days,1.33,4,0
